## Import Things

In [1]:
import gpt_2_simple as gpt2
from big_phoney import BigPhoney
import numpy as np
import lyricsgenius as genius
import pandas as pd

import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action
# model_name = "124M"
# model is saved into current directory under /models/124M/
# gpt2.download_gpt2(model_name=model_name) #need to run only once. comment out once done.

/Users/stasianik/opt/anaconda3/envs/insight/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/stasianik/opt/anaconda3/envs/insight/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/stasianik/opt/anaconda3/envs/insight/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/stasianik/op

## Load models and outside functions

In [2]:
#Initiate TensorFlow session, needs GPU
#Might need to start this inside the function so I can choose the right "run" a.k.a model
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

#Load text augmentation model
aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="insert")

#For syllable counting
phoney = BigPhoney()

Loading checkpoint checkpoint/run1/model-500
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-500


## Functions 

These will go in a Utils.py file 

In [35]:
def get_target_lyrics(artist, title):
    
    api_key = 'Nc3VjqRNlEZiD9mqOQATDQ5PuZ4IjdgV13n7c39OIyNwsjuazZThQlQkdr_Hts4c'
    api = genius.Genius(api_key)
    api.remove_section_headers = True # Removes section headers like "Verse" and "Intro"
    
    #get song lyrics
    song = api.search_song(artist, title)
    
    #clean up the target lyrics
    target = song.lyrics
    target = target.replace("\u2005"," ")
    target = target.replace("\\"," ")
    target = target.replace("\\n"," ")
    target = target.replace("("," ")
    target = target.replace(")"," ")
    target = target.replace("\n\n","\n")
    target = target.replace("\n\n\n","\n")
    target = target.replace("2x","")
    #create list of lines
    target_lyrics = target.split("\n")
    print("Target lyrics:")
    print(target_lyrics)
    return target_lyrics

In [4]:
# Later on I'll need to add the artist as a parameter, if I have 3, so it can load in the right finetuned model
# Figure out prefix
def generate_lyrics(): 
    
    gen_lyrics = gpt2.generate(sess, 
                     length=250,
                     temperature=0.9,
                     prefix="Deep in the night",
                     nsamples=1,
                     batch_size=1,
                     return_as_list=True)[0]
    
    gen_lyrics = gen_lyrics.split("\n")
    
    return gen_lyrics

In [11]:
def count_syls(text):
    
    schema = []
    
    for line in text:
         syls = phoney.count_syllables(line)
         schema.append(syls)
         #print(syls,line)
        
    return schema

#### The magical lyrics fitting function that hopefully works consistently!

In [6]:
#This function uses the count_syls function above to keep track of num of syllables in each line
def fit_lyrics(gen_lyrics, target_lyrics):
    
       gen_schema = count_syls(gen_lyrics)
    target_schema = count_syls(target_lyrics)
    
    #make generated lyrics same length as target lyrics
    target_len = len(target_schema)
    del gen_schema[target_len:]
    del gen_lyrics[target_len:]

    print(target_schema)
    new_lyrics = []
    
    for num, line in enumerate(gen_lyrics):
        print(num)
        if (gen_schema[num] == target_schema[num]): 
            new_lyrics.append(line)
        elif (gen_schema[num] < target_schema[num]):
            syls = gen_schema[num]
            while syls < target_schema[num]:
                line = aug.augment(line)
                syls = phoney.count_syllables(line)
                #In case we overshoot (add too many syllables)
                if syls > target_schema[num]:
                    print("Oops we overshot")
                print("---------")
                print("syls:")
                print(syls)
                target = target_schema[num]
                print("target:")
                print(target)
                print(num)
            new_lyrics.append(line)
        elif (gen_schema[num] > target_schema[num]):
            syls = gen_schema[num]
            words = line.split(" ")
            while syls > target_schema[num]:
                del words[-1]
                new_line = ' '.join(words)
                syls = phoney.count_syllables(new_line)
                #In case too many syllables are deleted
                if syls < target_schema[num]:
                    print("Oops, deleted too many")
                    new_line = aug.augment(new_line)
                    syls = phoney.count_syllables(new_line)
                    #print(syls)
                #print(syls)
            new_lyrics.append(new_line)
    print("New lyrics:")
    print(new_lyrics)
    
    return new_lyrics

## Time to test everything

In [36]:
artist = "Idina Menzel"
title = "Let it go"
target_lyrics = get_target_lyrics(artist, title)

Searching for "Idina Menzel" by Let it go...
Done.
Target lyrics:
['The snow glows white on the mountain tonight', 'Not a footprint to be seen', 'A kingdom of isolation', "And it looks like I'm the queen", 'The wind is howling like this swirling storm inside', "Couldn't keep it in, Heaven knows I tried", "Don't let them in, don't let them see", 'Be the good girl you always have to be', "Conceal, don't feel, don't let them know", 'Well, now they know', 'Let it go, let it go', "Can't hold it back anymore", 'Let it go, let it go', 'Turn away and slam the door', "I don't care what they're going to say", 'Let the storm rage on!', 'The cold never bothered me anyway', "It's funny how some distance", 'Makes everything seem small', 'And the fears that once controlled me', "Can't get to me at all", "It's time to see what I can do", 'To test the limits and break through', 'No right, no wrong, no rules for me', "I'm free", 'Let it go, let it go', "I'm one with the wind and sky", 'Let it go, let it

In [8]:
gen_lyrics = generate_lyrics()

In [16]:
print(target_lyrics)

['The snow glows white on the mountain tonight', 'Not a footprint to be seen', 'A kingdom of isolation', "And it looks like I'm the queen", 'The wind is howling like this swirling storm inside', "Couldn't keep it in, Heaven knows I tried Don't let them in, don't let them see", 'Be the good girl you always have to be', "Conceal, don't feel, don't let them know", 'Well, now they know Let it go, let it go', "Can't hold it back anymore", 'Let it go, let it go', 'Turn away and slam the door', "I don't care what they're going to say", 'Let the storm rage on!', "The cold never bothered me anyway It's funny how some distance", 'Makes everything seem small', 'And the fears that once controlled me', "Can't get to me at all It's time to see what I can do", 'To test the limits and break through', 'No right, no wrong, no rules for me', "I'm free Let it go, let it go", "I'm one with the wind and sky", 'Let it go, let it go', "You'll never see me cry", "Here I stand, and here I'll stay", 'Let the sto

In [ ]:
fit_lyrics(gen_lyrics, target_lyrics)

In [39]:
gen_schema = count_syls(gen_lyrics)
target_schema = count_syls(target_lyrics)
    
    #make generated lyrics same length as target lyrics
target_len = len(target_schema)
del gen_schema[target_len:]
del gen_lyrics[target_len:]

print(target_schema)
new_lyrics = []
    
for num, line in enumerate(gen_lyrics):
    print(num)
    if (gen_schema[num] == target_schema[num]): 
        new_lyrics.append(line)
    elif (gen_schema[num] < target_schema[num]):
        syls = gen_schema[num]
        while syls < target_schema[num]:
            line = aug.augment(line)
            syls = phoney.count_syllables(line)
            #In case we overshoot (add too many syllables)
            if syls > target_schema[num]:
                print("Oops we overshot")
                    #print(syls)
                    #print("For line:")
                    #print(num)
                print("---------")
            print("syls:")
            print(syls)
            target = target_schema[num]
            print("target:")
            print(target)
            print(num)
        new_lyrics.append(line)
    elif (gen_schema[num] > target_schema[num]):
        syls = gen_schema[num]
        words = line.split(" ")
        while syls > target_schema[num]:
            del words[-1]
            new_line = ' '.join(words)
            syls = phoney.count_syllables(new_line)
            #In case too many syllables are deleted
            if syls < target_schema[num]:
                print("Oops, deleted too many")
                new_line = aug.augment(new_line)
                syls = phoney.count_syllables(new_line)
                #print(syls)
            #print(syls)
        new_lyrics.append(new_line)
print("New lyrics:")
print(new_lyrics)

[10, 7, 8, 7, 12, 10, 8, 10, 8, 4, 6, 7, 6, 7, 9, 5, 10, 7, 6, 8, 6, 8, 8, 8, 2, 6, 7, 6, 6, 7, 5, 12, 14, 12, 12, 6, 8, 6, 6, 8, 5, 10]
0
1
2
syls:
8
target:
8
2
3
4
5
Oops we overshot
---------
syls:
12
target:
10
5
6
7
syls:
10
target:
10
7
8
syls:
7
target:
8
8
syls:
8
target:
8
8
9
10
11
12
13
14
syls:
8
target:
9
14
Oops we overshot
---------
syls:
11
target:
9
14
15
Oops, deleted too many
Oops, deleted too many
16
Oops we overshot
---------
syls:
11
target:
10
16
17
18
19
syls:
6
target:
8
19
syls:
7
target:
8
19
syls:
8
target:
8
19
20
Oops, deleted too many
21
syls:
4
target:
8
21
syls:
7
target:
8
21
syls:
8
target:
8
21
22
syls:
6
target:
8
22
syls:
8
target:
8
22
23
syls:
5
target:
8
23
syls:
6
target:
8
23
syls:
7
target:
8
23
syls:
8
target:
8
23
24
25
26
syls:
5
target:
7
26
syls:
6
target:
7
26
syls:
7
target:
7
26
27
syls:
3
target:
6
27
syls:
3
target:
6
27
syls:
3
target:
6
27
syls:
4
target:
6
27
Oops we overshot
---------
syls:
7
target:
6
27
28
syls:
4
target:
6
2

In [40]:
print(target_lyrics)

['The snow glows white on the mountain tonight', 'Not a footprint to be seen', 'A kingdom of isolation', "And it looks like I'm the queen", 'The wind is howling like this swirling storm inside', "Couldn't keep it in, Heaven knows I tried", "Don't let them in, don't let them see", 'Be the good girl you always have to be', "Conceal, don't feel, don't let them know", 'Well, now they know', 'Let it go, let it go', "Can't hold it back anymore", 'Let it go, let it go', 'Turn away and slam the door', "I don't care what they're going to say", 'Let the storm rage on!', 'The cold never bothered me anyway', "It's funny how some distance", 'Makes everything seem small', 'And the fears that once controlled me', "Can't get to me at all", "It's time to see what I can do", 'To test the limits and break through', 'No right, no wrong, no rules for me', "I'm free", 'Let it go, let it go', "I'm one with the wind and sky", 'Let it go, let it go', "You'll never see me cry", "Here I stand, and here I'll stay

In [38]:
print(target_schema)

[10, 7, 8, 7, 12, 10, 8, 10, 8, 4, 6, 7, 6, 7, 9, 5, 10, 7, 6, 8, 6, 8, 8, 8, 2, 6, 7, 6, 6, 7, 5, 12, 14, 12, 12, 6, 8, 6, 6, 8, 5, 10]
